# 1 Time Series 7-Day Forecasting with multi-layer perceptron<a id='1_time_series_7-day_forecasting_with_mlp'></a>

## 1.1 Contents<a id='1.1_contents'></a>
* [1 Covid-19 Data Collection and Wrangling](#1_data_collection_wrangling)
  * [1.1 Contents](#1.1_contents)  
  * [1.2 Imports](#1.2_imports)
  * [1.3 Functions](#1.3_functions)
      * [1.3.1 Function: ](#1.3.1_)

In [2]:
import pickle
from pathlib import Path
from datetime import datetime
from isoweek import Week
import math

In [3]:
# load the saved dictionary from pickle file
filePath_pickle = Path('/Users/parkj/Documents/pyDat/dataSet/covid_country_data.pickle')
with open(filePath_pickle, 'rb') as f:
    dict_country = pickle.load(f)

In [51]:
def feature_list(dict_country): 
    for i, country_key in enumerate(dict_country.keys()):
        df_country = dict_country[country_key]
        for row in df_country.iterrows():
            dt = row[0].to_pydatetime()
            year = dt.year
            month = dt.month
            day = dt.day
            week_of_year = dt.isocalendar()[1]
            day_of_week = row[1].dayow
            
    
    return 